# 

# <font size =8> Regularização

A regularização é uma técnica em machine learning utilizada para melhorar a generalização dos modelos, prevenindo o overfitting. Overfitting ocorre quando um modelo se ajusta muito bem aos dados de treinamento, mas falha em generalizar para novos dados. A regularização adiciona um termo de penalização à função de custo do modelo, que desencoraja a complexidade excessiva e ajuda a manter os coeficientes do modelo sob controle.

É amplamente utilizada em diversos tipos de modelos de machine learning, especialmente em regressões lineares e logísticas, para melhorar a robustez e a capacidade de generalização. Ela é aplicada em situações onde:

* **Há muitas características**: Para evitar overfitting em datasets com um grande número de características, especialmente quando algumas delas podem ser irrelevantes ou redundantes.
* **Multicolinearidade**: Quando as características estão correlacionadas, a regularização ajuda a estabilizar as estimativas dos coeficientes.
* **Interpretação do Modelo**: Em problemas onde a interpretabilidade é importante, a regularização pode ajudar a identificar as características mais relevantes.

Em resumo, a regularização é uma ferramenta poderosa para melhorar a performance dos modelos de machine learning, tornando-os mais robustos e interpretáveis, especialmente em cenários com dados complexos e de alta dimensionalidade.

---

## <font size=6>Imports

In [1]:
#autoreload
%load_ext autoreload
%autoreload 2

In [2]:
# Warnings
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Imports
import numpy as np
import pandas as pd

from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso, Ridge, ElasticNet
from sklearn.model_selection import train_test_split, cross_val_score

# utils
from utils.eda_utils import remove_acentuacao

---

# <font size=8> Regressors

# Load Data

In [4]:
# load
df = pd.read_csv('data/cholesterol.csv')

In [5]:
# Preview
df

,Id,Grupo Sanguíneo,Fumante,Nível de Atividade,Idade,Peso,Altura,Colesterol
0,1,B,Sim,Baixo,33.0,85.1,186.0,199.63
1,2,A,Não,Moderado,68.0,105.0,184.0,236.98
2,3,O,Não,Alto,25.0,64.8,180.0,161.79
3,4,A,Não,Alto,43.0,120.2,167.0,336.24
4,5,AB,Não,Baixo,79.0,88.5,175.0,226.23
...,...,...,...,...,...,...,...,...
995,996,AB,Sim,Moderado,31.0,68.1,166.0,206.81
996,997,O,Não,Alto,51.0,47.7,170.0,128.03
997,998,AB,Não,Baixo,39.0,85.5,176.0,211.14
998,999,AB,Sim,Baixo,61.0,91.2,161.0,284.53


# Processing

In [6]:
# drop id
df.drop(columns='Id', inplace=True)

# rename columns
df.columns = [col.lower() for col in df.columns]

# replace white spaces
df.columns = [col.replace(' ', '_') for col in df.columns]

# remove accent
df.columns = [remove_acentuacao(col) for col in df.columns]

# dropna
df.dropna(inplace=True)

# One Hot encode
df = pd.get_dummies(df, columns=['grupo_sanguineo', 'fumante', 'nivel_de_atividade'])

# Set target
target = 'colesterol'

In [7]:
# View
df

,idade,peso,altura,colesterol,grupo_sanguineo_A,grupo_sanguineo_AB,grupo_sanguineo_B,grupo_sanguineo_O,fumante_Não,fumante_Sim,nivel_de_atividade_Alto,nivel_de_atividade_Baixo,nivel_de_atividade_Moderado
0,33.0,85.1,186.0,199.63,0,0,1,0,0,1,0,1,0
1,68.0,105.0,184.0,236.98,1,0,0,0,1,0,0,0,1
2,25.0,64.8,180.0,161.79,0,0,0,1,1,0,1,0,0
3,43.0,120.2,167.0,336.24,1,0,0,0,1,0,1,0,0
4,79.0,88.5,175.0,226.23,0,1,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,31.0,68.1,166.0,206.81,0,1,0,0,0,1,0,0,1
996,51.0,47.7,170.0,128.03,0,0,0,1,1,0,1,0,0
997,39.0,85.5,176.0,211.14,0,1,0,0,1,0,0,1,0
998,61.0,91.2,161.0,284.53,0,1,0,0,0,1,0,1,0


In [8]:
# Separar os dados
X = df.drop(target, axis=1)
y = df[target]

# Separar treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=51)

# Functions

In [9]:
# Performance
def performance_model(model, X_test, y_test):
    
    # Prediction
    y_pred = model.predict(X_test)
    
    return mean_squared_error(y_test, y_pred, squared=False)

In [10]:
# Criar um pipeline para escalonar os dados e ajustar o modelo
def create_pipeline(model):
    return Pipeline([
        ('scaler', StandardScaler()),
        ('regressor', model)
    ])

# Models

In [11]:
# Features names
feature_names = X_train.columns

In [12]:
# Definir os modelos
lasso = create_pipeline(Lasso(alpha=0.1, random_state=42))
ridge = create_pipeline(Ridge(alpha=1.0, random_state=42))
elastic_net = create_pipeline(ElasticNet(alpha=0.1, l1_ratio=0.5, random_state=42))

In [13]:
# Avaliar os modelos usando validação cruzada
models = {'Lasso': lasso, 'Ridge': ridge, 'ElasticNet': elastic_net}
results = {}

for name, model in models.items():
    scores = cross_val_score(model, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
    results[name] = np.mean(scores)
    print(f"{name} MSE: {-np.mean(scores):.4f}")

Lasso MSE: 78.9743
Ridge MSE: 79.3403
ElasticNet MSE: 92.8183


---

# Regularização L1 (Lasso)

### Características:
   * Ênfase na seleção de features. Quanto maior o alpha, maior será a regularizção e mais coeficientes tendem a ser reduzidos a zero.
   * Útil para fins de feature selection e para modelos esparsos. Adiciona uma penalidade à função de custo proporcional à soma absoluta dos valores dos pesos.

### Vantagens:
   * Pode induzir esparsidade nos pesos, levando a modelos mais simples e interpretáveis.
   * Útil quando há suspeita de que muitos dos recursos de entrada são irrelevantes.

### Desvantagens:
   * Pode tornar a otimização mais desafiadora devido a função não possuir uma derivada definida, pois não há inclinação suave.

In [14]:
# Ajustar o modelo Lasso para inspecionar os coeficientes
lasso.fit(X_train, y_train)
lasso_coef = lasso.named_steps['regressor'].coef_

lasso_coef_series = pd.Series(lasso_coef, index=feature_names)
lasso_coef_sorted = lasso_coef_series.reindex(lasso_coef_series.abs().sort_values(ascending=False).index)

print("\nLasso Coefficients:\n")
print(lasso_coef_sorted)


Lasso Coefficients:

peso                           52.779403
altura                        -19.448690
grupo_sanguineo_AB              1.214534
fumante_Não                    -1.115419
nivel_de_atividade_Alto        -0.534324
nivel_de_atividade_Baixo        0.283418
idade                          -0.080303
grupo_sanguineo_A              -0.032644
grupo_sanguineo_B              -0.000000
grupo_sanguineo_O               0.000000
fumante_Sim                     0.000000
nivel_de_atividade_Moderado    -0.000000
dtype: float64


In [15]:
# Avalirar o modelo
print(f'MSE: {performance_model(lasso, X_test, y_test)}')

MSE: 8.971879743621065


---

# Regularização L2 (Ridge)

### Características:
* Ênfase na redução de variância e generalização do modelo (reduz overfitting).
* Útil principalmente quando há multicolinearidade. Adiciona uma penalidade à função de custo proporcional à soma dos quadrados dos valores dos pesos.

###  Vantagens:
* Ajuda a evitar pesos muito grandes, promovendo modelos mais suaves e menos sensíveis a pequenas * variações nos dados (Variância).
* Facilita a otimização, pois é diferenciável em todos os lugares.

###  Desvantagens:
* Não induz esparsidade nos pesos.

In [16]:
# Ajustar o modelo Ridge para inspecionar os coeficientes
ridge.fit(X_train, y_train)
ridge_coef = ridge.named_steps['regressor'].coef_

ridge_coef_series = pd.Series(ridge_coef, index=feature_names)
ridge_coef_sorted = ridge_coef_series.reindex(ridge_coef_series.abs().sort_values(ascending=False).index)

print("\nRidge Coefficients:\n")
print(ridge_coef_sorted)


Ridge Coefficients:

peso                           52.784899
altura                        -19.518348
grupo_sanguineo_AB              0.964885
fumante_Sim                     0.600941
fumante_Não                    -0.600941
nivel_de_atividade_Alto        -0.531738
grupo_sanguineo_A              -0.428186
nivel_de_atividade_Baixo        0.415457
grupo_sanguineo_O              -0.278051
grupo_sanguineo_B              -0.275259
idade                          -0.178941
nivel_de_atividade_Moderado     0.069490
dtype: float64


In [17]:
# Avalirar o modelo
print(f'MSE: {performance_model(ridge, X_test, y_test)}')

MSE: 8.99212979268138


---

# Regularização Elastic Net

### Características:
* Combina as penalidades L1 (Lasso) e L2 (Ridge).
* Útil quando há muitas features correlacionadas ou quando se deseja uma combinação de seleção de features e regularização.

### Vantagens:
* Flexibilidade para ajustar a proporção de penalidades L1 e L2, permitindo um equilíbrio entre seleção de features e redução de variância.
* Pode melhorar o desempenho em dados onde Lasso ou Ridge sozinhos não são suficientes.

### Desvantagens:
* Requer a escolha de dois hiperparâmetros (um para cada tipo de penalidade), o que pode aumentar a complexidade do ajuste do modelo.
* Pode ser computacionalmente mais intensivo devido à necessidade de ajustar múltiplos parâmetros.

In [18]:
# Ajustar o modelo Elastic Net para inspecionar os coeficientes
elastic_net.fit(X_train, y_train)
elastic_net_coef = elastic_net.named_steps['regressor'].coef_

elastic_net_coef_series = pd.Series(elastic_net_coef, index=feature_names)
elastic_net_coef_sorted = elastic_net_coef_series.reindex(elastic_net_coef_series.abs().sort_values(ascending=False).index)

print("\nElastic Net Coefficients:\n")
print(elastic_net_coef_sorted)


Elastic Net Coefficients:

peso                           48.454125
altura                        -17.012498
fumante_Não                    -1.239679
grupo_sanguineo_AB              1.178263
fumante_Sim                     1.151963
nivel_de_atividade_Alto        -1.018730
grupo_sanguineo_O              -0.953656
nivel_de_atividade_Baixo        0.891105
grupo_sanguineo_B              -0.423553
idade                          -0.239601
grupo_sanguineo_A               0.055217
nivel_de_atividade_Moderado    -0.000000
dtype: float64


In [19]:
# Avalirar o modelo
print(f'MSE: {performance_model(elastic_net, X_test, y_test)}')

MSE: 9.507076548120105


---

# <font size=8> Classifiers

# Imports

In [23]:
# Imports
from sklearn.linear_model import LogisticRegression

# Load Data

In [24]:
# load
df = pd.read_csv('data/fruit_quality.csv')

In [25]:
# Preview
df

,A_id,Size,Weight,Sweetness,Crunchiness,Juiciness,Ripeness,Acidity,Quality
0,0,-3.970049,-2.512336,5.346330,-1.012009,1.844900,0.329840,-0.491590,good
1,1,-1.195217,-2.839257,3.664059,1.588232,0.853286,0.867530,-0.722809,good
2,2,-0.292024,-1.351282,-1.738429,-0.342616,2.838636,-0.038033,2.621636,bad
3,3,-0.657196,-2.271627,1.324874,-0.097875,3.637970,-3.413761,0.790723,good
4,4,1.364217,-1.296612,-0.384658,-0.553006,3.030874,-1.303849,0.501984,good
...,...,...,...,...,...,...,...,...,...
3995,3995,0.059386,-1.067408,-3.714549,0.473052,1.697986,2.244055,0.137784,bad
3996,3996,-0.293118,1.949253,-0.204020,-0.640196,0.024523,-1.087900,1.854235,good
3997,3997,-2.634515,-2.138247,-2.440461,0.657223,2.199709,4.763859,-1.334611,bad
3998,3998,-4.008004,-1.779337,2.366397,-0.200329,2.161435,0.214488,-2.229720,good


# Processing

In [26]:
# drop id
df.drop(columns='A_id', inplace=True)

# rename columns
df.columns = [col.lower() for col in df.columns]

# replace white spaces
df.columns = [col.replace(' ', '_') for col in df.columns]

# remove accent
df.columns = [remove_acentuacao(col) for col in df.columns]

# dropna
df.dropna(inplace=True)

# Target
target = 'quality'
df['quality'] = (df['quality'] == 'good').astype(int)

In [27]:
# View
df

,size,weight,sweetness,crunchiness,juiciness,ripeness,acidity,quality
0,-3.970049,-2.512336,5.346330,-1.012009,1.844900,0.329840,-0.491590,1
1,-1.195217,-2.839257,3.664059,1.588232,0.853286,0.867530,-0.722809,1
2,-0.292024,-1.351282,-1.738429,-0.342616,2.838636,-0.038033,2.621636,0
3,-0.657196,-2.271627,1.324874,-0.097875,3.637970,-3.413761,0.790723,1
4,1.364217,-1.296612,-0.384658,-0.553006,3.030874,-1.303849,0.501984,1
...,...,...,...,...,...,...,...,...
3995,0.059386,-1.067408,-3.714549,0.473052,1.697986,2.244055,0.137784,0
3996,-0.293118,1.949253,-0.204020,-0.640196,0.024523,-1.087900,1.854235,1
3997,-2.634515,-2.138247,-2.440461,0.657223,2.199709,4.763859,-1.334611,0
3998,-4.008004,-1.779337,2.366397,-0.200329,2.161435,0.214488,-2.229720,1


In [28]:
# Separar os dados
X = df.drop('quality', axis=1)
y = df['quality']

# Separar treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=51)

---

# Models

In [31]:
# Features names
feature_names = X_train.columns

In [29]:
# Definir os modelos
lasso = create_pipeline(LogisticRegression(penalty='l1', C=0.1, solver='liblinear', random_state=42))
ridge = create_pipeline(LogisticRegression(penalty='l2', C=0.1, solver='liblinear', random_state=42))

In [30]:
# Avaliar os modelos usando validação cruzada
models = {'Lasso': lasso, 'Ridge': ridge}
results = {}

for name, model in models.items():
    scores = cross_val_score(model, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
    results[name] = np.mean(scores)
    print(f"{name} MSE: {-np.mean(scores):.4f}")

Lasso MSE: 0.2582
Ridge MSE: 0.2618


---

# Regularização L1 (Lasso)

### Características:
   * Ênfase na seleção de features. Quanto maior o alpha, maior será a regularizção e mais coeficientes tendem a ser reduzidos a zero.
   * Útil para fins de feature selection e para modelos esparsos. Adiciona uma penalidade à função de custo proporcional à soma absoluta dos valores dos pesos.

### Vantagens:
   * Pode induzir esparsidade nos pesos, levando a modelos mais simples e interpretáveis.
   * Útil quando há suspeita de que muitos dos recursos de entrada são irrelevantes.

### Desvantagens:
   * Pode tornar a otimização mais desafiadora devido a função não possuir uma derivada definida, pois não há inclinação suave.

In [34]:
# Ajustar o modelo Lasso para inspecionar os coeficientes
lasso.fit(X_train, y_train)
lasso_coef = lasso.named_steps['regressor'].coef_[0]

lasso_coef_series = pd.Series(lasso_coef, index=feature_names)
lasso_coef_sorted = lasso_coef_series.reindex(lasso_coef_series.abs().sort_values(ascending=False).index)

print("\nLasso Coefficients:\n")
print(lasso_coef_sorted)


Lasso Coefficients:

size           1.085051
sweetness      0.992598
juiciness      0.774423
acidity       -0.524495
weight         0.349329
ripeness      -0.233747
crunchiness    0.000000
dtype: float64


In [35]:
# Avalirar o modelo
performance_model(lasso, X_test, y_test)

0.47696960070847283

---

# Regularização L2 (Ridge)

### Características:
* Ênfase na redução de variância e generalização do modelo (reduz overfitting).
* Útil principalmente quando há multicolinearidade. Adiciona uma penalidade à função de custo proporcional à soma dos quadrados dos valores dos pesos.

###  Vantagens:
* Ajuda a evitar pesos muito grandes, promovendo modelos mais suaves e menos sensíveis a pequenas * variações nos dados (Variância).
* Facilita a otimização, pois é diferenciável em todos os lugares.

###  Desvantagens:
* Não induz esparsidade nos pesos.

In [32]:
# Ajustar o modelo Ridge para inspecionar os coeficientes
ridge.fit(X_train, y_train)
ridge_coef = ridge.named_steps['regressor'].coef_[0]

ridge_coef_series = pd.Series(ridge_coef, index=feature_names)
ridge_coef_sorted = ridge_coef_series.reindex(ridge_coef_series.abs().sort_values(ascending=False).index)

print("\nRidge Coefficients:\n")
print(ridge_coef_sorted)


Ridge Coefficients:

size           1.092517
sweetness      0.999361
juiciness      0.792077
acidity       -0.545598
weight         0.365329
ripeness      -0.245057
crunchiness    0.021616
dtype: float64


In [33]:
# Avalirar o modelo
performance_model(ridge, X_test, y_test)

0.4752192476461084

---